A good tutorial video: https://www.youtube.com/watch?v=vmEHCJofslg

In [ ]:
df = pd.read_cvs('fn.txt',delimiter=',')


In [ ]:
# read headers
df.columns 

# read one column by its index name
df['<Name of the column>']

# read one row
df.head(10) 
df.tail(10)
# read column by index
df.iloc[0]
# find column by the name of the rows
df.loc[df['Name'] == 'Lin']

# read info in each row; returns index of the row + content of the row
df.iterrows()
for index, row in df.iterrows():
    print(index, row['Name'])

# sorting
df.sort_values()

# groupby 
df.groupby(['Type 1']).mean().sort_values('Type 2', ascending=False)


In [1]:
import numpy as np
import xarray as xr
import pandas as pd


In [3]:
ds = pd.read_csv('ERA5_OMI_daily_Oct18.txt', sep=',', header=0)
ds 

,Date,PC1,PC2
0,1979-01-01,0.09520,0.66056
1,1979-01-02,-0.03654,0.70443
2,1979-01-03,-0.20001,0.74170
3,1979-01-04,-0.38933,0.76571
4,1979-01-05,-0.59552,0.76996
...,...,...,...
14970,2019-12-27,0.14528,-0.62493
14971,2019-12-28,0.06852,-0.54444
14972,2019-12-29,-0.01017,-0.43375
14973,2019-12-30,-0.08608,-0.30017


In [8]:
fr = '/global/homes/l/linyaoly/ERA5/reanalysis/realOMI.txt'
dr = pd.read_fwf(fr, infer_nrows=400, names=['Year','Month','Day','PC1','PC2','Amp'],sep=' ')
dr = dr[0:14975]
dr 

,Year,Month,Day,PC1,PC2,Amp
1979,1,1,0,0.16495,0.77777,0.79507
1979,1,2,0,0.02067,0.85471,0.85496
1979,1,3,0,-0.15643,0.91998,0.93319
1979,1,4,0,-0.35961,0.96423,1.02910
1979,1,5,0,-0.57882,0.97869,1.13704
...,...,...,...,...,...,...
2019,12,27,0,0.39191,-0.69125,0.79462
2019,12,28,0,0.33367,-0.55385,0.64660
2019,12,29,0,0.24763,-0.35787,0.43519
2019,12,30,0,0.13650,-0.11126,0.17610


In [25]:
ds['PC2'].sum(skipna=False)

47.293390000000024

In [26]:
pc1p = dr['PC1'].values
pc2p = dr['PC2'].values

pc1t = ds['PC1'].values
pc2t = ds['PC2'].values

a = sum(pc1p*pc1t+pc2p*pc2t)
b = np.sqrt(sum(pc1t*pc1t+pc2t*pc2t))
c = np.sqrt(sum(pc1p*pc1p+pc2p*pc2p))
BCC = a/b/c 
BCC 

0.9766975620648223

In [1]:
import numpy as np
import xarray as xr
import pandas as pd

dirn = '/pscratch/sd/l/linyaoly/ERA5/reanalysis/'
flg = 'sst'
ds1 = xr.open_mfdataset(dirn+flg+'_2deg*.nc',combine='nested',concat_dim="time")

ds1=ds1.rename({"longitude":"lon","latitude":"lat"})

ds1=ds1.resample(time="1D").mean('time')

# ds1['time'] = pd.to_datetime(ds1.time)

ds1.to_netcdf(dirn+'ERA5.'+flg+'.day.1978to2022.nc')
del ds1

In [4]:
import numpy as np
import xarray as xr
import pandas as pd

# remove climatology daily mean during 1979-2014
dirn = '/pscratch/sd/l/linyaoly/ERA5/reanalysis/'
flg = 'sst'
varn = 'sst'
ds = xr.open_dataset(dirn+'ERA5.'+flg+'.day.1978to2022.nc')
dstmp = ds.sel(time=slice('1979-01-01','2014-12-31'))
ds2 = dstmp.groupby("time.dayofyear").mean()

x = ds[varn].groupby("time.dayofyear") - ds2[varn]
ds[varn].values = x.values 
ds = ds.sel(time=slice('1979-01-01','2022-05-31'))
ds 

<xarray.Dataset>
Dimensions:  (time: 15857, lon: 180, lat: 91)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2022-05-31
  * lon      (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float32 90.0 88.0 86.0 84.0 82.0 ... -84.0 -86.0 -88.0 -90.0
Data variables:
    sst      (time, lat, lon) float32 0.0005798 0.0005798 0.0005798 ... nan nan

In [5]:
ds.sst.max()

<xarray.DataArray 'sst' ()>
array(17.22680664)

In [7]:
dstmp = ds.sel(time=slice('1979-01-01','2014-12-31'))
dstmp 

<xarray.Dataset>
Dimensions:  (time: 13149, lon: 180, lat: 91)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2014-12-31
  * lon      (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float32 90.0 88.0 86.0 84.0 82.0 ... -84.0 -86.0 -88.0 -90.0
Data variables:
    sst      (time, lat, lon) float32 0.0005798 0.0005798 0.0005798 ... nan nan

In [8]:
# normalized sst 
msst = dstmp['sst'].mean(dim=None, skipna=True)
msst

<xarray.DataArray 'sst' ()>
array(3.846779e-06, dtype=float32)

In [9]:
stdsst = dstmp['sst'].std(dim=None, skipna=True)

In [10]:
stdsst

<xarray.DataArray 'sst' ()>
array(0.56930977)

In [13]:
ds['sst'].values = (ds['sst'].values -  msst.values) / stdsst.values 
ds['sst']

<xarray.DataArray 'sst' (time: 15857, lat: 91, lon: 180)>
array([[[ 1.0117290e-03,  1.0117290e-03,  1.0117290e-03, ...,
          1.0117290e-03,  1.0117290e-03,  1.0117290e-03],
        [ 1.0117290e-03,  1.0117290e-03,  1.0117290e-03, ...,
          1.0117290e-03,  1.0117290e-03,  1.0117290e-03],
        [ 1.0117290e-03,  1.0653335e-03,  1.0653335e-03, ...,
          1.0117290e-03,  1.0117290e-03,  1.0117290e-03],
        ...,
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan]],

       [[-6.0361435e-05, -6.0361435e-05, -6.0361435e-05, ...,
         -6.0361435e-05, -6.0361435e-05, -6.0361435e-05],
        [-6.0361435e-05, -6.0361435e-05, -6.0361435e-05, ...,
         -6.0361435e-05, -6.0361435e-05, -6.0361435e-05],
        [-6.0361435e-05,  4.6847606e-05,  4.6847606e-05, ...,
         -6.0361435e-05, -6.0361435e-05, -6.0361435e-05],
...
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan]],

       [[ 1.1725426e-03,  1.1725426e-03,  1.1725426e-03, ...,
          1.1725426e-03,  1.1725426e-03,  1.1725426e-03],
        [ 1.1725426e-03,  1.1725426e-03,  1.1725426e-03, ...,
          1.1725426e-03,  1.1725426e-03,  1.1725426e-03],
        [ 1.1725426e-03,  1.1725426e-03,  1.1725426e-03, ...,
          1.1725426e-03,  1.1725426e-03,  1.1725426e-03],
        ...,
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2022-05-31
  * lon      (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float32 90.0 88.0 86.0 84.0 82.0 ... -84.0 -86.0 -88.0 -90.0

In [15]:
ds = ds.fillna(0)


In [16]:
ds 

<xarray.Dataset>
Dimensions:  (time: 15857, lat: 91, lon: 180)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2022-05-31
  * lon      (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float32 90.0 88.0 86.0 84.0 82.0 ... -84.0 -86.0 -88.0 -90.0
Data variables:
    sst      (time, lat, lon) float32 0.001012 0.001012 0.001012 ... 0.0 0.0 0.0

In [18]:
flg

'sst'

In [19]:
ds.to_netcdf(dirn+'ERA5.'+flg+'GfltGmask0.day.1979to2022.nc')

In [1]:
import numpy as np
import xarray as xr
import pandas as pd

dirn = '/pscratch/sd/l/linyaoly/ERA5/reanalysis/'
flg = 'sst'
ds1 = xr.open_mfdataset(dirn+flg+'_2deg*.nc',combine='nested',concat_dim="time")

ds1=ds1.rename({"longitude":"lon","latitude":"lat"})

ds1[flg].astype(float)

flg = 'sst'
varn = 'sst'
ds = ds1
dstmp = ds.sel(time=slice('1979-01-01','2014-12-31'))
ds2 = dstmp.groupby("time.dayofyear").mean()

x = ds[varn].groupby("time.dayofyear") - ds2[varn]
ds[varn].values = x.values 
ds = ds.sel(time=slice('1979-01-01','2022-05-31'))

dstmp = ds.sel(time=slice('1979-01-01','2014-12-31'))

# normalized sst 
msst = dstmp['sst'].mean(dim=None, skipna=True)
stdsst = dstmp['sst'].std(dim=None, skipna=True)
ds['sst'].values = (ds['sst'].values -  msst.values) / stdsst.values 
ds = ds.fillna(0)
ds.to_netcdf(dirn+'ERA5.'+flg+'GfltGmask0.6hr.1979to2022.nc')



In [3]:
xr.open_dataset('RMM_ERA5_6hr.nc')

<xarray.Dataset>
Dimensions:  (time: 63428, mode: 2)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 ... 2022-05-31T18:00:00
  * mode     (mode) int64 0 1
Data variables:
    RMM      (time, mode) float32 ...

In [1]:
import numpy as np
import xarray as xr
import pandas as pd

dirn = '/pscratch/sd/l/linyaoly/ERA5/reanalysis/'
flg = 'tcwv'
varn = 'tcwv'
ds1 = xr.open_mfdataset(dirn+flg+'_2deg*.nc',combine='nested',concat_dim="time")

ds1=ds1.rename({"longitude":"lon","latitude":"lat"})

# ds1[flg].astype(float)

# ds1['time'] = pd.to_datetime(ds1.time)

ds1.to_netcdf(dirn+'ERA5.'+flg+'.6hr.1978to2022.nc')


In [12]:
# del ds1 

In [2]:
ds = xr.open_dataset(dirn+'ERA5.'+flg+'.6hr.1978to2022.nc')
# 

In [3]:
ds 

<xarray.Dataset>
Dimensions:  (lon: 180, lat: 91, time: 64888)
Coordinates:
  * lon      (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float32 90.0 88.0 86.0 84.0 82.0 ... -84.0 -86.0 -88.0 -90.0
  * time     (time) datetime64[ns] 1978-01-01 ... 2022-05-31T18:00:00
Data variables:
    tcwv     (time, lat, lon) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-10-11 20:04:46 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [4]:
x = ds['tcwv']

In [5]:
# x.isnull()

In [6]:
np.sum(x.isnull()).values

array(17)